<a href="https://colab.research.google.com/github/WashingtonKE/Renewable-Energy-Optimization/blob/main/Whisper_Transcription_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎧 Whisper Transcription – Google Colab Notebook

This notebook lets you transcribe audio (e.g., `.aac`, `.mp3`, `.wav`, `.m4a`) using [OpenAI Whisper](https://github.com/openai/whisper) in Google Colab.

## How to use
1. **Run each cell in order** (Shift+Enter).
2. Upload your audio file when prompted.
3. The transcript will be printed and saved as `transcript.txt` (and optionally `transcript.srt`).

**Tip:** Change `MODEL_SIZE` to `small`, `medium`, or `large` for higher accuracy (needs more GPU memory/time).


In [1]:
# 🛠️ Step 1: Install Whisper and FFmpeg
!pip -q install openai-whisper
!sudo apt -yq update && !sudo apt -yq install ffmpeg
print("Installation complete.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://cli.github.com/packages stable InRelease
Hit:11 https://ppa.launchpadcontent.net/graphi

In [2]:
# 🎚️ Step 2: (Optional) Choose model size and language
MODEL_SIZE = "base"  # options: tiny, base, small, medium, large
FORCE_LANGUAGE = None  # e.g., "en" for English, "sw" for Swahili; set to None to auto-detect
print(f"Model size set to: {MODEL_SIZE}")
print(f"Force language: {FORCE_LANGUAGE}")

Model size set to: base
Force language: None


In [5]:
# 📤 Step 3: Upload your audio file
from google.colab import files
uploaded = files.upload()
assert uploaded, "No file uploaded. Please run this cell again and select an audio file."
FILENAME = list(uploaded.keys())[0]
print("Uploaded:", FILENAME)

Saving WhatsApp Audio 2025-11-02 at 1.15.56 AM.aac to WhatsApp Audio 2025-11-02 at 1.15.56 AM.aac
Uploaded: WhatsApp Audio 2025-11-02 at 1.15.56 AM.aac


In [6]:
# 🤖 Step 4: Transcribe with Whisper
import whisper, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = whisper.load_model(MODEL_SIZE, device=device)
kwargs = {}
if FORCE_LANGUAGE:
    kwargs["language"] = FORCE_LANGUAGE

result = model.transcribe(FILENAME, **kwargs)
text = result.get("text", "").strip()
print("\n--- TRANSCRIPTION ---\n")
print(text if text else "(No speech recognized or empty transcript)")

Using device: cpu


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 114MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



--- TRANSCRIPTION ---

I have been working as a data scientist now for about three years. And in this video I want to shut some light and explain the brutal truth behind what it's actually like being an scientist. Let's get into it. So what actually is a data scientist? Well, nowadays it can mean a range of things, but I like to think of it as a data scientist is someone who uses programming, maths, statistics and data to help solve business problems and provide insight into where things can be improved or how to use data more efficiently. To me, the main unique skill of data scientist is that of building machine learning models and the good statistical models. We do these to help drive business decision making and make process a lot more efficient than they would if they were just operated by humans. Common problems that data scientists may work on is predicting if a credit card payment was forage, lonesome odds, recommending the right products to certain customers on e-commerce webs

In [7]:
# 💾 Step 5: Save transcript and subtitles, then download
from google.colab import files

# Save plain text transcript
txt_name = "transcript.txt"
with open(txt_name, "w", encoding="utf-8") as f:
    f.write(result.get("text", "").strip())
print("Saved:", txt_name)

# Create simple SRT subtitles from segments
def to_srt(segments, path):
    def fmt(t):
        h = int(t // 3600)
        m = int((t % 3600) // 60)
        s = int(t % 60)
        ms = int((t - int(t)) * 1000)
        return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"
    with open(path, "w", encoding="utf-8") as srt:
        for i, seg in enumerate(segments, start=1):
            start = fmt(seg.get("start", 0.0))
            end = fmt(seg.get("end", 0.0))
            text = seg.get("text", "").strip()
            srt.write(f"{i}\n{start} --> {end}\n{text}\n\n")

srt_name = "transcript.srt"
to_srt(result.get("segments", []), srt_name)
print("Saved:", srt_name)

# Download files
files.download(txt_name)
files.download(srt_name)

Saved: transcript.txt
Saved: transcript.srt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>